## Vertical regridding example
This notebook requires `intake`, `intake-esm` and `gcsfs` which are *not* dependencies of pyIC: you will need to install these.

This notebook is based on the `xgcm` example for regridding: https://xgcm.readthedocs.io/en/latest/transform.html#Realistic-Data-Example.

In [ ]:
import cf_xarray
import intake
import numpy as np
import xarray as xr

from pyic.grid import GRID

In [ ]:
# use intake to get some cmip6 data

col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

cat = col.search(
    source_id="CNRM-ESM2-1",
    member_id="r1i1p1f2",
    experiment_id="historical",
    variable_id=["thetao", "so", "vo", "areacello"],
    grid_label="gn",
)
ddict = cat.to_dataset_dict(zarr_kwargs={"consolidated": True, "use_cftime": True}, aggregate=False)

In [ ]:
# construct datasets from cmip data

thetao = ddict[
    "CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r1i1p1f2.Omon.thetao.gn.gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/historical/r1i1p1f2/Omon/thetao/gn/v20181206/.20181206"
]
so = ddict[
    "CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r1i1p1f2.Omon.so.gn.gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/historical/r1i1p1f2/Omon/so/gn/v20181206/.20181206"
]
vo = ddict[
    "CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r1i1p1f2.Omon.vo.gn.gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/historical/r1i1p1f2/Omon/vo/gn/v20181206/.20181206"
]
areacello = ddict[
    "CMIP.CNRM-CERFACS.CNRM-ESM2-1.historical.r1i1p1f2.Ofx.areacello.gn.gs://cmip6/CMIP6/CMIP/CNRM-CERFACS/CNRM-ESM2-1/historical/r1i1p1f2/Ofx/areacello/gn/v20181206/.20181206"
].areacello

vo = vo.rename(
    {"y": "y_c", "lon": "lon_v", "lat": "lat_v", "bounds_lon": "bounds_lon_v", "bounds_lat": "bounds_lat_v"}
)

ds = xr.merge([thetao, so, vo], compat="override")
ds = ds.assign_coords(areacello=areacello.fillna(0))
ds

In [ ]:
# assign coords

level_outer_data = cf_xarray.bounds_to_vertices(ds.lev_bounds, "axis_nbounds").load().data

ds = ds.assign_coords({"level_outer": level_outer_data})

In [ ]:
# use vertical regrid in pyIC to produce regridded vertical

In [ ]:
ds_rg = GRID(
    dataset=ds,
    convert_to_z_grid=True,
    z_kwargs={
        "variable": "all",
        "target": np.arange(0, 500.0, 50),
        # "target_variable":"thetao",
        "coord": "lev",
        "method": "linear",
    },
)

In [ ]:
# regridded
ds_rg.ds["so"]

In [ ]:
# original
ds_rg.raw_ds["so"]